In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'f:\\pythoncode\\ML\\ML Projects\\Text-Summarization-using-NLP'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path
    tokenizer_name : Path

In [5]:
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config
    

In [6]:
from text_summarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_en = self.tokenizer(example_batch['dialogue'], truncation=True, max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_en = self.tokenizer(example_batch['summary'], truncation=True, max_length=128)

        return {
            'input_ids': input_en['input_ids'],
            'attention_mask': input_en['attention_mask'],
            'labels': target_en['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e


[2024-10-28 18:00:51,354: INFO: common: Successfully loaded YAML data from config\config.yaml]
[2024-10-28 18:00:51,358: INFO: common: Successfully loaded YAML data from params.yaml]
[2024-10-28 18:00:51,361: INFO: common: Created directory: artifacts]
[2024-10-28 18:00:51,362: INFO: common: Created directory: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]f:\pythoncode\ML\ML Projects\Text-Summarization-using-NLP\myvenv\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 116869.59 examples/s]
